# Scalable Diffusion Models with Transformers

참고링크
+ https://kimjy99.github.io/%EB%85%BC%EB%AC%B8%EB%A6%AC%EB%B7%B0/dit/

## Contributions
+ 기존의 U-Net backbone을 Transformer로 변경하였다.
+ 모델을 scaling하며 sample quality와 Gflops간의 상관 관계를 연구하였다.
+ SOTA 달성하였다

__추가로, 요즘 Sora, stable diffusion v3 와 같은 최신모델들이 Diffusion transformer를 사용하고 있다.__

## 1. Introduction


+ 지금은 Transformer의 르네상스 시대를 맞고있다.
    + 그런데 DDPM의 백본으로 사용하는 U-Net을 보면 CNN기반에 self-attention추가한 정도이다.
    + 거기에서 구조의 변경도 크게 없다.

+ 우리는 백본으로 Transformer 모델을 사용해보았다.
    + 우리는 생성모델에 있어서 U-Net의 inductive bias가 의미가 없는 것을 보였다.(CNN vs ViT에서의 논쟁)
    + 이러한 구조의 모델을 DiT라고 부르겠다.
    + 이 모델의 scaling을 통해 모델 complexity(Gflops) vs 샘플 quality의 trade off를 연구하였다.

## 2. Preliminaries

### Diffusion formulation

+ forward process  
미리 hyper parameter로 scheduling되어있는 variance값 $\alpha$로부터 sampling할 수 있다. (by reparameterization trick)  

 $x_0 : q(x_t|x_T) = \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t}x_0, (1-{\bar{\alpha}_t})\mathbf{I})$

+ reverse process  
neural net $\theta$를 통해 forward process의 역과정을 예측한다.  
$p_\theta (x_{t-1}|x_t) = \mathcal{N}(\mu_\theta(x_t), \Sigma_\theta(x_t))$

+ loss function  
Variational lower bound를 최대화하는 방향으로 학습한다.  
$\mathcal{L}(\theta) = -p(x_0|x_1) + \sum_t{\mathcal{D}_{KL}(q^*(x_{t-1}|x_t, x_0)||p_\theta(x_{t-1}|x_t))}$  
    + 이 때, $\mu_\theta$ 대신 $\epsilon_\theta$를 출력하는 모델을 이용해 학습시킨다.
    + $\epsilon_\theta$는 $\mathcal{L}_{simple}$을 학습시키고 $\Sigma_\theta$는 $\mathcal{L}$전체를 학습시킨다. (Improved ddpm paper)

### Classifier-free guidance

+ Classifier guidance(Diffusion models beats GANs paper)  
Condition diffusion 모델은 결국 sampling에서  
$\log p(c|x)$를 최대화하는 x를 찾는 것이다.  
그러한 모델 $p_\phi(c|x)$를 정의한다.

+ Classifier-free guidance  
기존 conditional loss의 $\log p(c|x)$ 텀을 베이지안 룰을 통해 잘 변경하였더니, 따로 모델을 사용할 필요가 없어졌다.  
$\hat{\epsilon_\theta}(x_t, c) = \epsilon_\theta(x_t, \emptyset) + s \cdot (\epsilon_\theta(x_t, c) - \epsilon_\theta(x_t, \emptyset))$  
    + $c = \emptyset$ -> "null" embedding
    + s : cfg hyper parameter

### Latent diffusion models

기존의 DDPM process를 2-stage로 나누었다.
+ learnning autoencoder with compressed images representations $E(\cdot)$
+ encoder는 고정시켜놓고 $z = E(x)$를 diffusion model에 학습시킨다. 그리고 decoder를 학습한다. $x = D(z)$

__본 논문에서는 classifier free guidance, convolution 기반의 VAE, transformer 기반 DDPM을 사용한다__

## 3. Diffusion Transformer Design Space

<img src="https://drive.google.com/uc?id=10HmVQYk1nqzO0oDFR8a4O1k1FIklZqyk">

### Patchify
ViT의 patchify방식을 사용하였다.  
각각의 patch size는 $p \times p \times C$이다.

<img src="https://drive.google.com/uc?id=1_nCHqnKk-r51QRwDJJx6iEeWug-D4QVA" height=300>

patchify이후, 모든 input token들에 ViT와 마찬가지로 frequency-based positional embedding을 사용.(sine-cosine)

p값에 따라서 inuput sequence 길이가 정해지는데, 이는 모델의 파라미터 수에는 큰 영향을 주지 않고, Gflops(계산 수)에는 상당한 영향을 미친다.($n^2$)  

p 값을 2, 4, 8로 적용해보았다.

### DiT block design

diffusion 모델에서 condition 정보(t : time step, c : class labels 등)를 넣는 것은 중요하다. 여러 방법으로 디자인해보았다.

+ In-context conditioning  
    ViT에서 cls token과 비슷하게, input sequence에 두개의 추가 token으로 embedding한다.  
    마지막 output에서 해당 token들을 제거한다.

+ Cross-attention block  
    embedding한 t, c를 길이 2짜리의 sequence로 concate해놓고 이번엔 image token과 cross-attention을 해준다.

+ Adaptive layer norm (adaLN) block  
    layer norm 이후에 t,c token을 통해 scale & shift 해준다. 이 때 regress 모델을 통해 해당 factor를 예측시킨다.

+ adaLN-Zero block  
    + 기존 연구에서 residual block의 경우 마지막 conv block에 대해서 zero-initializing이 학습을 가속화 해준다는 연구가 있다.  
    + 이와 마찬가지로 모든 DiT block의 residual connection 이전에 dimension-wise scaling 파라미터를 적용하였다. 그리고,  MLP를 통과하고 나온 해당 파라미터가 zero-vector가 되도록 initialize하였다.

__즉, $\textrm{MLP}(\textrm{embed}(t, c)) = \vec\gamma, \vec\beta, \vec\alpha$ 이고,
$\alpha$는 0이 되도록 초기화__  
그리고 이 때의 block은 Gflops상 무시할만한 크기이다.

### Model size


<img src="https://drive.google.com/uc?id=1qeghfPbBKzVPLjFFKLMQT-g53VXl_UPT" height=200>

ViT에서와 마찬가지로 scaling하기 위해 N, d, h를 사용하였다. 이에 따라 Gflops가 변화하였다.

모델 사이즈에 따라서 0.3 ~ 118.3까지의 변화가 있었다.

### Transformer decoder

layer norm하고 각 token에 대해서 $p\times p\times2C$로 linearly decoding을 해준다.

그리고, 해당 output을 각각 원래 image위치로 재배열하고 noise와 covariance($\epsilon, \Sigma$)를 예측하는데에 사용한다.

## 4. Experimental Setup

### Training

+ Diffusion setup
    + ADM(Diffusion model beats GANs)에서 사용한 training setting을 그대로 사용하였다.  
    + VAE는 Stable Diffusion을 그대로 사용하였다.

+ DiT setup
    + DiT모델을 training 할 때에는 보통 ViT할 때와 달리 warmup scaduler, regularization을 딱히 사용하지 않아도 안정적이어서 그렇게 했다고한다.  
    + 훈련간에 model weight에대해 EMA를 적용하였다고 한다.

### Evaluation
FID-50k를 주요 지표로 삼았다.  
이외에도 sFID, IS, Precision/Recall에 대해서도 실험하였다

## 5. Expriments

### DiT block design / Scaling model and patch size

+ adaLN-Zero block이 다른 것들에 비해 낮은 compute로 좋은 FID score를 기록하였다. 앞으로 이 블럭을 사용한다.
+ model configs : S, B, L, XL
+ patch sizes : 8, 4, 2

총 12개의 모델을 통해 분석해본다.

### DiT Gflops are critical to improving performance

<img src="https://drive.google.com/uc?id=1qYe9d8OGYVSkyN7-zoIGZm-gX1KYPTye" height=300>

+ model과 patch 사이즈를 적절히 변경했을 때 Gflops와 FID점수가 명확히 log scale의 상관관계를 가진다.  
+ S/2와 B/4모델을 보면 각각의 파라미터 수는 다른데도 patch크기로 인해 동일한 Gflops를 갖게 되었고 FID점수 역시 동일하였다.  


이러한 경향성은 이후에 다른 실험에서도 동일하게 나타난다.

### Larger DiT models are more compute-efficient

<img src="https://drive.google.com/uc?id=1Yftdg7Vmu3XqZlcxHOAB1FYWg7F8ei7i" height=350>

Training Compute = Gflops x batch size x training steps x 3

+ 결국엔 compute efficiency면에서 작은 model을 long step training하더라도 더 큰 model의 적은 step에 비해 별로다.
+ patch가 크면, 적은 Gflops로도 더 좋은 성능을 보이는 경향을 보인다. 하지만, 일정 step이 넘어가면 결국 더 작은 patch일 때가 좋다.

### Visualizing scaling

<img src="https://drive.google.com/uc?id=1kVCY-_9M4y6EJ8GPJHvcNSNPSoPBJBcW" height=400>

### SOTA Diffusion
<img src="https://drive.google.com/uc?id=15xSjydrBTwo6yKFmcC-YxoXo0DO-SFDz" height=350>
<img src="https://drive.google.com/uc?id=1yWStRlIr7z-dKrcZACE7A8SK2s3osKyI" height=250>

256 size에서는 모든 생성모델에 대해 SOTA,   
512 size에서는 Diffusion 모델중에서 SOTA

### Scaling Model vs Sampling Compute

<img src="https://drive.google.com/uc?id=1K8tVXKj6WJPBBtPnHxnd9tFVcfEzeSwp" height=350>

각 모델에 대해서 [16, 32, 64, 128, 256, 1000]으로 sampling steps를 바꿔가며 FID를 측정해보았다.

+ L/2(1000 steps)와 XL/2(128 steps)를 비교해보았을 때, 비슷한 퀄리티에비해 연산량은 5배정도 차이가 난다.
+ 즉, sampling step이 model scaling에 비해 큰 영향을 미치지는 않는다.

## 결론

Model backbone을 U-Net에서 Transformer 으로 바꾸어 scale관점에서 체계적으로 연구한 의의가있다.